In [241]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

# Metodo Random Forest

In [277]:
df = pd.read_csv("./train.csv")


In [278]:
def mean_target_encoding(train, nombrefeat, nombretarget):
        
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean) #Saco promedio
    temp = preprocessing.scale(temp) #Normalizo
    train[nombrefeat + "_mean"] = temp #Dejo en el DF
    
    return train


In [279]:
def mean_target_decoding(x_test, nombrefeat, x_train):
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 

    return x_test

In [284]:
df['agregados']= df['escuelascercanas']+df['gimnasio']+df['centroscomercialescercanos']+df['piscina']+df['usosmultiples']
train = df.drop(['id','idzona','lat','direccion','descripcion','titulo','lng', 'escuelascercanas','gimnasio','centroscomercialescercanos','piscina','usosmultiples'],1)
train['fecha'] = pd.to_datetime(train['fecha']).dt.year
train.fillna(0, inplace=True)
X = train
y = train['precio']

from sklearn.model_selection import train_test_split
#Separo los datos de "train" en entrenamiento y prueba para probar los algoritmos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

#X_train = mean_target_encoding(X_train, "tipodepropiedad", "precio")
X_train = mean_target_encoding(X_train, "provincia", "precio")
X_train = mean_target_encoding(X_train, "ciudad", "precio")

X_test = mean_target_decoding(X_test, "provincia", X_train)
#X_test = mean_target_decoding(X_test, "tipodepropiedad", X_train)
X_test = mean_target_decoding(X_test, "ciudad", X_train)

X_train = X_train.drop(['precio','tipodepropiedad','provincia','ciudad'],1) 
X_train.fillna(0, inplace=True)
X_test = X_test.drop(['precio','tipodepropiedad','provincia','ciudad'],1) 
X_test.fillna(0, inplace=True)

/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [285]:
rf_random.best_params_

NameError: name 'rf_random' is not defined

In [288]:
# Crear el modelo 
forest = RandomForestRegressor(n_estimators= 74, min_samples_split=10,min_samples_leaf=4,max_features= "sqrt",max_depth=42,bootstrap = False, verbose=2)
#Lo entreno 
forest.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 74
building tree 2 of 74
building tree 3 of 74
building tree 4 of 74
building tree 5 of 74
building tree 6 of 74
building tree 7 of 74
building tree 8 of 74
building tree 9 of 74
building tree 10 of 74
building tree 11 of 74
building tree 12 of 74
building tree 13 of 74
building tree 14 of 74
building tree 15 of 74
building tree 16 of 74
building tree 17 of 74
building tree 18 of 74
building tree 19 of 74
building tree 20 of 74
building tree 21 of 74
building tree 22 of 74
building tree 23 of 74
building tree 24 of 74
building tree 25 of 74
building tree 26 of 74
building tree 27 of 74
building tree 28 of 74
building tree 29 of 74
building tree 30 of 74
building tree 31 of 74
building tree 32 of 74
building tree 33 of 74
building tree 34 of 74
building tree 35 of 74
building tree 36 of 74
building tree 37 of 74
building tree 38 of 74
building tree 39 of 74
building tree 40 of 74
building tree 41 of 74
building tree 42 of 74
building tree 43 of 74
building tree 44 of 

[Parallel(n_jobs=1)]: Done  74 out of  74 | elapsed:    5.5s finished


RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=42,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=74,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=2, warm_start=False)

In [289]:
Y_pred_forest = forest.predict(X_test)
print('MAE: ', metrics.mean_absolute_error(y_test, Y_pred_forest))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


MAE:  657315.953644291


[Parallel(n_jobs=1)]: Done  74 out of  74 | elapsed:    2.7s finished


## Regresion lineal

In [272]:
 #Defino el algoritmo a utilizar
regresion_lineal = linear_model.LinearRegression()
#Entreno el modelo
regresion_lineal.fit(X_train, y_train)

    
#Realizo una predicción
Y_pred_resgresion = regresion_lineal.predict(X_test)

print('DATOS DEL MODELO REGRESIÓN LINEAL MULTIPLE')
print('Valor de las pendientes o coeficientes "a":')
print(regresion_lineal.coef_)
print('Valor de la intersección o coeficiente "b":')
print(regresion_lineal.intercept_)


DATOS DEL MODELO REGRESIÓN LINEAL MULTIPLE
Valor de las pendientes o coeficientes "a":
[ -9584.4278303  -21140.02357942 -37512.45253943 176662.74685424
   7233.12815357   3265.68949201 142396.62910729  42918.16491669
 211416.20126535 220162.62182318 831383.3917511 ]
Valor de la intersección o coeficiente "b":
-286182697.55872494


In [273]:
print('Precisión del modelo:')
print(regresion_lineal.score(X_train, y_train))

Precisión del modelo:
0.5098635284794366


In [274]:
print('MAE: ', metrics.mean_absolute_error(y_test, Y_pred_resgresion))

MAE:  1012544.7831299728


## Regresion + forest

In [275]:
solucion = ( Y_pred_resgresion+Y_pred_forest)/2
solucion

array([3026722.11909799, 2123115.93241471, 5752193.19206432, ...,
       2518846.16075811, 3225861.17448787, 1695116.70936667])

In [276]:
print('MAE: ', metrics.mean_absolute_error(y_test, solucion))

MAE:  830775.062637502


In [ ]:
#El mae da muy bajo, revisar que no se este filtrando info con el mean de los feature